In [ ]:
!pip install ../input/efficientnet-install-files/Ker*.whl
!pip install ../input/efficientnet-install-files/eff*.whl

In [ ]:
import os
import glob
import json
import numpy as np
import keras
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as k
from PIL import Image
import matplotlib.pyplot as plt
from keras import backend as K 
import efficientnet.keras as efn
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

cwd = os.getcwd()
os.chdir('../input/bitempered-logistic-loss-direct-upload/')
from tf_bi_tempered_loss import BiTemperedLogisticLoss
os.chdir(cwd)

# remove warnings (most of them are not critical and meant for debugging tensorflow)
import warnings
warnings.filterwarnings("ignore")
loss = BiTemperedLogisticLoss(t1=0.6, t2=1.4)

In [ ]:
models = []
image_size = 512
n_splits=4
# model_name = 'effnetb4'
# fold_name = '-fold.h5'
# cwd = os.getcwd()
# os.chdir('../input/newcassavadataset')
# for i in range(n_splits):
#     effnetb4 = keras.models.load_model(model_name + str(i+1) + fold_name,compile=False)
#     models.append(effnetb4)
# os.chdir(cwd)

eff_models = os.listdir('../input/from-the-tpu-notebook')
for i in eff_models:
    print(i)
    models.append(keras.models.load_model(i))

# models.append(keras.models.load_model('../input/newcassavadataset/effnetb01-fold.h5'))
print(models)
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)

results = []

for image_id in test_images:
    preds = []
    image = Image.open(os.path.join('../input/cassava-leaf-disease-classification', "test_images", image_id))
    image = image.resize((image_size, image_size))
    image = np.expand_dims(image, axis = 0)
    for model in models:
        p = model.predict(image)
        preds.append(np.argmax(p))
        print(p)
    res = max(set(preds), key = preds.count)
    results.append(res)

sub = pd.DataFrame({'image_id': test_images, 'label': results})

In [ ]:
display(sub)
sub.to_csv('submission.csv', index = False)

In [ ]:
# results

In [ ]:
# TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
# image_size = 512
# n_splits=3
# test_images = os.listdir(TEST_DIR)
# datagen = ImageDataGenerator(horizontal_flip=True)
# final_model = keras.models.load_model('../input/newcassavadataset/effnetb41-fold.h5',compile=False)


# for image in test_images:
#     K.clear_session()
#     img = Image.open(TEST_DIR + image)
#     img = img.resize((image_size,image_size))
#     samples = np.expand_dims(img, axis=0)
#     it = datagen.flow(samples, batch_size=1)
#     yhats = final_model.predict_generator(it, verbose=1)
#     summed = np.sum(yhats, axis=0)
# predictions = np.argmax(summed)

# # predictions = pred(test_images)
# sub = pd.DataFrame({'image_id': test_images, 'label': predictions})
# display(sub)
# sub.to_csv('submission.csv', index = False)

# # tta_steps = 10
# predictions = []

# for i in tqdm(range(tta_steps)):
#     preds = model.predict_generator(train_datagen.flow(x_val, batch_size=bs, shuffle=False), steps = len(x_val)/bs)
#     predictions.append(preds)

# pred = np.mean(predictions, axis=0)

# np.mean(np.equal(np.argmax(y_val, axis=-1), np.argmax(pred, axis=-1)))